In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
train_spg = np.load('./trainset_spg.npy')
test_spg = np.load('./testset_spg.npy')
val_spg = np.load('./valset_spg.npy')

In [3]:
train_data = np.load('trainset_values.npy')
test_data = np.load('testset_values.npy')
val_data = np.load('valset_values.npy')

In [5]:
data = test_spg
np.max(data), np.min(data)

(230, 1)

In [4]:
train_data = np.expand_dims(train_data, axis=2)
test_data = np.expand_dims(test_data, axis=2)
val_data = np.expand_dims(val_data, axis=2)

In [7]:
train_data.shape

(114404, 10001, 1)

In [5]:
train_Y230 = np.load('./trainset_spg.npy')-1
test_Y230 = np.load('./testset_spg.npy')-1
val_Y230 = np.load('./valset_spg.npy')-1

In [6]:
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [7]:
train_Y230 = to_categorical(train_Y230, 230)
test_Y230 = to_categorical(test_Y230, 230)
val_Y230 = to_categorical(val_Y230, 230)

In [11]:
train_Y230.shape

(114404, 230)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold

In [13]:
#X_train, X_test, y_train, y_test = train_test_split(train_data, train_Y230, test_size=0.2, random_state=42)

In [9]:
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, merge, Input
from keras.layers import Dropout, Activation, Bidirectional
from keras.layers import Convolution1D, MaxPooling1D, AveragePooling1D, LSTM
from keras.layers import ZeroPadding1D
from keras.layers.noise import GaussianNoise
from keras.optimizers import SGD
import keras.callbacks
from keras.models import Model
from keras_self_attention import SeqSelfAttention

In [10]:
model = Sequential()

model.add(Convolution1D(80, 100, subsample_length = 5, border_mode = 'same', input_shape=(10001,1))) #add convolution layer
model.add(Activation('relu')) #activation
model.add(Dropout(0.3))
model.add(AveragePooling1D(pool_length=3, stride=2)) #pooling layer
model.add(Convolution1D(80, 50, subsample_length = 5, border_mode = 'same'))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(AveragePooling1D(pool_length=3, stride=None))
model.add(Convolution1D(80, 25, subsample_length = 2, border_mode ='same'))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(AveragePooling1D(pool_length=3, stride=None))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(AveragePooling1D(pool_length=3, stride=None))
#model.add(Flatten())

model.add(Bidirectional(LSTM(80,input_shape=(10001,1), return_sequences=True)))
#model.add(SeqSelfAttention(attention_activation='sigmoid'))
#model.add(Dropout(0.3))
#model.add(LSTM(80,input_shape=train_data.shape, return_sequences=False))
model.add(Flatten())
#model.add(LSTM(80, input_shape=X_train.shape,return_sequences=True))
model.add(Dense(2300))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1150))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(230))
model.add(Activation('softmax'))

#Compile
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()
#fit
filepath='C:\\Users\\Arun.Sagotra\\Desktop\\CNN-Models\\XRD\\CuKa-5-105-september\\Model\\model_lstm.out'
modelCheckpoint=keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto')
history = keras.callbacks.History()
model.fit(train_data, train_Y230, batch_size=500, epochs=2, validation_data= (val_data, val_Y230), callbacks=[modelCheckpoint,history])
# check the accuracy
a = model.evaluate(train_data, train_Y230)
print(a)

print("===== Test Data =====")
a = model.evaluate(test_data, test_Y230)
print(a)
#save log after training
acc_log = history.history['acc']
val_acc_log = history.history['val_acc']
loss_log = history.history['loss']
val_loss_log = history.history['val_loss']
acc_log = np.array(acc_log)
val_acc_log = np.array(val_acc_log)
loss_log = np.array(loss_log)
val_loss_log = np.array(val_loss_log)
mat = np.vstack((loss_log, acc_log, val_loss_log, val_acc_log))
mat = np.transpose(mat)
dataframe1 = pd.DataFrame(data=mat)
dataframe1.to_csv('save_log_lstm.csv', sep=',', header=False, float_format='%.7f', index=False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Users\Arun.Sagotra\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(80, 100, input_shape=(10001, 1), strides=5, padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Arun.Sagotra\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `AveragePooling1D` call to the Keras 2 API: `AveragePooling1D(pool_size=3, strides=2)`
  
C:\Users\Arun.Sagotra\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(80, 50, strides=5, padding="same")`
  import sys
C:\Users\Arun.Sagotra\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `AveragePooling1D` call to the Keras 2 API: `AveragePooling1D(pool_size=3, strides=None)`
  # Remove the CWD from sys.path while we load 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 2001, 80)          8080      
_________________________________________________________________
activation_1 (Activation)    (None, 2001, 80)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2001, 80)          0         
_________________________________________________________________
average_pooling1d_1 (Average (None, 1000, 80)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 200, 80)           320080    
_________________________________________________________________
activation_2 (Activation)    (None, 200, 80)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 200, 80)           0         
__________

KeyboardInterrupt: 

In [ ]:
data_acc= pd.read_csv('save_log_lstm.csv', sep=',', header=None)

In [ ]:
plt.plot(data_acc[1], label='Trainset')
plt.plot(data_acc[3], label='Testset')
plt.legend()

In [ ]:
history.history['acc']

In [ ]:
import torch

In [16]:
import segmentation_models_pytorch as smp